In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
xrange = range
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import os

#pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 200)

print("The working directory was: {0}".format(os.getcwd()))
os.chdir("C:/hjBoralex/etl/gitcwd")
print("The current working directory is: {0}".format(os.getcwd()))

The working directory was: C:\hjBoralex\etl
The current working directory is: C:\hjBoralex\etl\gitcwd


In [162]:
mypath = "C:/Users/hermann.ngayap/Desktop/data/"
marketpath = "C:/Users/hermann.ngayap/Desktop/data/market/markets/"

In [167]:
prod = pd.read_excel(marketpath + "template_prod_id.xlsx", sheet_name="prod")
prod_perc = pd.read_excel(marketpath + "template_prod_id.xlsx", sheet_name="prod_perc")
mean_perc = pd.read_excel(marketpath + "template_prod_id.xlsx", sheet_name="mean_perc")


df_asset = pd.read_excel(marketpath + "asset.xlsx")
df_asset = df_asset.loc[df_asset["en_planif"] == "Non"]

df_planif = pd.read_excel(marketpath + "asset.xlsx")
df_planif = df_planif.loc[df_planif["en_planif"] == "Oui"]

df_hedge = pd.read_excel(marketpath + "hedge.xlsx")
df_hedge_vmr = df_hedge.loc[df_hedge["en_planif"] == "Non"]

df_hedge_planif = df_hedge.loc[df_hedge["en_planif"] == "Oui"]


df1 = pd.read_excel(marketpath + "template_hedge.xlsx")

In [168]:
df1.tail()

,ID,projet_id,projet,technologie,type_hedge,date_debut,date_fin,date_dementelement,puissance_installée,profil,pct_couverture,contrepartie,pays_contrepartie,en_planif
291,196,GO10,Repowering Grand-Camp,éolien,CR,2025-03-01,2045-03-01,NaT,9.60000,NaN,1.0,NaN,NaN,Oui
292,197,BFC15,Prospection PV BFC - 6,solaire,CR,2027-09-01,2047-09-01,NaT,3.93984,NaN,1.0,NaN,NaN,Oui
293,198,NGE9,Repowering St-Patrick Valhuon,éolien,CR,2027-04-01,2047-04-01,NaT,22.46400,NaN,1.0,NaN,NaN,Oui
294,199,NGE19,XLCB 2,éolien,CR,2027-08-01,2047-08-01,NaT,7.12800,NaN,1.0,NaN,NaN,Oui
295,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.00000,NaN,1.0,Metro,France,Oui


In [101]:
# df_h_eol = df1.loc[(df["technologie"] == "éolien") & (df["en_planif"] == "Oui")]
# df_h_sol = df1.loc[(df["technologie"] == "solaire") & (df["en_planif"] == "Oui")]

# HEDGE PLANIF 

## HEDGE CR

In [172]:
#To fill n/a with 0.80
#df1["puissance_installée"].fillna(df1["puissance_installée"].mean(skipna=True), inplace=True)
#To calculate mw 
#df1["puissance_installée"] = df1["mw"] * df1["taux_succès"]

df1.loc[(df1['technologie'] == 'éolien') & (df1['en_planif'] == 'Oui'), 'p50'] = df1["puissance_installée"] * 8760 * 0.25#calculer le p_50 projet éolien en planification
df1.loc[(df1['technologie'] == 'éolien') & (df1['en_planif'] == 'Oui'), 'p90'] = df1["puissance_installée"] * 8760 * 0.20#calculer le p_90 projet éolien en planification

df1.loc[(df1['technologie'] == 'solaire') & (df1['en_planif'] == 'Oui'), 'p50'] = df1["puissance_installée"] * 8760 * 0.15#calculer le p_50 projet solaire en planification
df1.loc[(df1['technologie'] == 'solaire') & (df1['en_planif'] == 'Oui'), 'p90'] = df1["puissance_installée"] * 8760 * 0.13#calculer le p_90 projet solaire en planification


df1["p50"] = df1["p50"] * df1["pct_couverture"]
df1["p90"] = df1["p90"] * df1["pct_couverture"]

prod_planif = df1.copy()
prod_planif_solaire = prod_planif.loc[(prod_planif['technologie'] == "solaire") & (prod_planif['en_planif'] == 'Oui')]
prod_planif_eolien = prod_planif.loc[(prod_planif['technologie'] == "éolien") & (prod_planif['en_planif'] == 'Oui')]

prod_planif_solaire.reset_index(drop = True, inplace=True)
prod_planif_eolien.reset_index(drop = True, inplace=True)

#To determine the number of solar and eolien
n_sol = len(prod_planif_solaire)
n_eol = len(prod_planif_eolien)

#prod_planif_solaire.to_excel(marketpath + "prod_planif_hedge.xlsx", index = False)
#prod_planif_eolien.to_excel(marketpath + "prod_planif_hedge.xlsx", index = False)

mean_perc_sol = mean_perc.iloc[:,[0, 1]]
mean_perc_eol = mean_perc.iloc[:,[0,-1]]

#create a dataframe with date from 2022 to 2028 solaire
start_date = pd.to_datetime(["01-01-2022"] * n_sol)
d1 = pd.DataFrame()
for i in range(0,84):
    df_buffer = prod_planif_solaire.copy() 
    df_buffer["date"] = start_date
    d1 = pd.concat([d1, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)
    
#reset index    
d1.reset_index(drop=True, inplace=True)

#create a mth column containing number of month
mean_perc_sol = mean_perc_sol.assign(mth=[1 + i for i in xrange(len(mean_perc_sol))])[['mth'] + mean_perc_sol.columns.tolist()]

#To calculate p50 and p90 solar
s = mean_perc_sol.set_index('mth')['m_pct_solaire']
pct = pd.to_datetime(d1['date']).dt.month.map(s)
d1['p50_adj'] = -d1['p50'] * pct
d1['p90_adj'] = -d1['p90'] * pct

#create a dataframe with date from 2022 to 2028 éolien
start_date = pd.to_datetime(["01-01-2022"] * n_eol)
d2 = pd.DataFrame()
for i in range(0,84):
    df_buffer = prod_planif_eolien.copy() 
    df_buffer["date"] = start_date
    d2 = pd.concat([d2, df_buffer],axis=0)
    start_date = start_date + pd.DateOffset(months=1)

#reset index    
d2.reset_index(drop=True, inplace=True)

#create a mth column containing number of month
mean_perc_eol = mean_perc_eol.assign(mth=[1 + i for i in xrange(len(mean_perc_eol))])[['mth'] + mean_perc_eol.columns.tolist()]

#To calculate p50 and p90 eolien
s2 = mean_perc_eol.set_index('mth')['m_pct_eolien']
pct = pd.to_datetime(d2['date']).dt.month.map(s2)
d2['p50_adj'] = -d2['p50'] * pct
d2['p90_adj'] = -d2['p90'] * pct

In [179]:
d1.loc[(d1["projet_id"] == "SE19") & (d1["ID"] == 200)]

,ID,projet_id,projet,technologie,type_hedge,date_debut,date_fin,date_dementelement,puissance_installée,profil,pct_couverture,contrepartie,pays_contrepartie,en_planif,p50,p90,date,p50_adj,p90_adj
106,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2022-01-01,-888.47424,-770.011008
213,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2022-02-01,-1176.92352,-1020.000384
320,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2022-03-01,-1880.59680,-1629.850560
427,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2022-04-01,-2040.37920,-1768.328640
534,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2022-05-01,-2321.25984,-2011.758528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8559,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2028-08-01,-2448.45504,-2121.994368
8666,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2028-09-01,-1921.59360,-1665.381120
8773,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2028-10-01,-1443.92832,-1251.404544
8880,200,SE19,Pézènes-les-Mines,solaire,PPA,2024-01-01,2043-12-31,NaT,16.0,NaN,1.0,Metro,France,Oui,21024.0,18220.8,2028-11-01,-917.90784,-795.520128


In [181]:
#To create new columns année et mois
d1["date_debut"] = pd.to_datetime(d1["date_debut"])
d1["date_fin"] = pd.to_datetime(d1["date_fin"])
d1["date_dementelement"] = pd.to_datetime(d1["date_dementelement"])
d1['année'] = d1['date'].dt.year
d1['mois'] = d1['date'].dt.month
d1 = d1[['ID','projet_id', 'projet', 'date_debut', "date_fin", 'date_dementelement', 'date', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To remove p50 p90 based on date_debut
#Condition:date column is less (in total seconds) than first date for each projet_id's first date_debut value
cond=((d1['date'] - d1.groupby(['projet_id', 'ID'])['date_debut'].transform('first')).dt.total_seconds())<0
d1['p50_adj'] = np.where(cond,'', d1['p50_adj'])
d1['p90_adj'] = np.where(cond,'', d1['p90_adj'])
#To remove p50 p90 based on date_fin
cond_2=((d1['date'] - d1.groupby(['projet_id', 'ID'])['date_fin'].transform('first')).dt.total_seconds())>0
d1['p50_adj'] = np.where(cond_2, '', d1['p50_adj'])
d1['p90_adj'] = np.where(cond_2, '', d1['p90_adj'])
#To remove based on date_debut_2
# cond_3=((d1['date'] - d1.groupby('projet_id')['date_fin'].transform('first')).dt.total_seconds())>0
# d1['p50_adj'] = np.where(cond_3, '', d1['p50_adj'])
# d1['p90_adj'] = np.where(cond_3, '', d1['p90_adj'])
 

#To reset index
d1.reset_index(inplace=True, drop=False)
d1 = d1.assign(nber=[1 + i for i in xrange(len(d1))])[['nber'] + d1.columns.tolist()]
d1 = d1[['nber', 'ID', 'projet_id', 'projet', 'année', 'mois', 'p50_adj', 'p90_adj']]

################################################################################
################################################################################
###############################################################################

#To create new columns année et mois
d2["date_debut"] = pd.to_datetime(d2["date_debut"])
d2["date_fin"] = pd.to_datetime(d2["date_fin"])
d2['année'] = d2['date'].dt.year
d2['mois'] = d2['date'].dt.month
d2 = d2[['ID','projet_id', 'projet', 'date_debut', 'date_fin', 'date_dementelement', 'date', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To remove p50 p90 based on date_debut
#Condition:date column is less (in total seconds) than first date for each projet_id's first date_cod value
cond=((d2['date'] - d2.groupby(['projet_id', 'ID'])['date_debut'].transform('first')).dt.total_seconds())<0
d2['p50_adj'] = np.where(cond,'', d2['p50_adj'])
d2['p90_adj'] = np.where(cond,'', d2['p90_adj'])
#To remove p50 p90 based on date_dementelemnt
cond_2=((d2['date'] - d2.groupby(['projet_id', 'ID'])['date_fin'].transform('first')).dt.total_seconds())>0
d2['p50_adj'] = np.where(cond_2, '', d2['p50_adj'])
d2['p90_adj'] = np.where(cond_2, '', d2['p90_adj'])
#To remove p50 p90 based on date_fin
# cond_3=((d2['date'] - d2.groupby('projet_id')['date_fin'].transform('first')).dt.total_seconds())>0
# d2['p50_adj'] = np.where(cond_3, '', d2['p50_adj'])
# d2['p90_adj'] = np.where(cond_3, '', d2['p90_adj'])

#To reset index
d2.reset_index(inplace=True, drop=False)
d2 = d2.assign(nber=[1 + i for i in xrange(len(d2))])[['nber'] + d2.columns.tolist()]
d2 = d2[['nber','ID', 'projet_id', 'projet', 'année', 'mois', 'p50_adj', 'p90_adj']]

#To export as excel files
d1.to_excel(marketpath + 'hedge_planif_solaire.xlsx', index=False)
d2.to_excel(marketpath + 'hedge_planif_eolien.xlsx', index=False)

In [187]:
print(prod_planif.loc[(prod_planif['projet_id'] == 'SE19') & (prod_planif['ID'] == 72)])
df_temp = d1.loc[(d1['projet_id'] == 'SE19') & (d1['ID'] == 72)]
df_temp.head(20)

     ID projet_id             projet technologie type_hedge date_debut  \
167  72      SE19  Pézènes-les-Mines     solaire        PPA 2023-01-01   

      date_fin date_dementelement  puissance_installée  profil  \
167 2023-12-31                NaT                 16.0     NaN   

     pct_couverture contrepartie pays_contrepartie en_planif     p50      p90  
167             0.4         Axpo            France       Oui  8409.6  7288.32  


,nber,ID,projet_id,projet,année,mois,p50_adj,p90_adj
28,29,72,SE19,Pézènes-les-Mines,2022,1,,
135,136,72,SE19,Pézènes-les-Mines,2022,2,,
242,243,72,SE19,Pézènes-les-Mines,2022,3,,
349,350,72,SE19,Pézènes-les-Mines,2022,4,,
456,457,72,SE19,Pézènes-les-Mines,2022,5,,
563,564,72,SE19,Pézènes-les-Mines,2022,6,,
670,671,72,SE19,Pézènes-les-Mines,2022,7,,
777,778,72,SE19,Pézènes-les-Mines,2022,8,,
884,885,72,SE19,Pézènes-les-Mines,2022,9,,
991,992,72,SE19,Pézènes-les-Mines,2022,10,,


#### MERGE SOLAIRE EOLIEN

In [202]:
df_1=pd.read_excel(marketpath + 'hedge_planif_solaire.xlsx')
df_2=pd.read_excel(marketpath + 'hedge_planif_eolien.xlsx')
print(df_1.shape)
print(df_2.shape)

(8988, 8)
(7812, 8)


In [203]:
frames = [df_1, df_2]
hedge_planif =pd.concat(frames, axis= 0, ignore_index=True)
#hedge_planif.reset_index(drop=True, inplace=True)
hedge_planif.to_excel(marketpath + "hedge_planif_sol_eol.xlsx", index=False)

#### MERGE HEDGE

In [207]:
df_3=pd.read_excel(marketpath + 'hedge_vmr.xlsx')
df_4=pd.read_excel(marketpath + 'hedge_planif_sol_eol.xlsx')
print(df_3.shape)
print(df_4.shape)

(8064, 8)
(16800, 8)


In [209]:
frames = [df_3, df_4]
hedge =pd.concat(frames, axis= 0, ignore_index=True)
#hedge_planif.reset_index(drop=True, inplace=True)
hedge.to_excel(marketpath + "hedge_vmr_planif.xlsx", index=False)

In [211]:
hedge.tail()

,nber,ID,projet_id,projet,année,mois,p50_adj,p90_adj
24859,7808.0,192,GO19,Repowering Ronchois,2028,12,-9828.541296,-7862.833037
24860,7809.0,195,SO14,Bruyeres - Glenic,2028,12,-3033.500400,-2426.800320
24861,7810.0,196,GO10,Repowering Grand-Camp,2028,12,-2426.800320,-1941.440256
24862,7811.0,198,NGE9,Repowering St-Patrick Valhuon,2028,12,-5678.712749,-4542.970199
24863,7812.0,199,NGE19,XLCB 2,2028,12,-1801.899238,-1441.519390
